### Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from skimage.data import imread
from pathlib import Path
from tqdm import tqdm, tqdm_notebook
import cv2
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#### Load train.csv

In [ ]:
train = pd.read_csv("../input/train.csv", 
                    index_col = 0)
print(train.shape)
train.head()

In [ ]:
haveMask = ~(train.rle_mask.isna())
haveMask.sum()

### Visualize 8 images with pixels having salt

In [ ]:
samples = train[haveMask].sample(8)
tgs = "../input"
tgs1 = "../input/tgs-salt-identification-challenge"
fig, ax = plt.subplots(4, 4, sharex= "col", sharey = "row")
fig.set_size_inches(12, 12)
for i, imgId in enumerate(samples.index):
    row = (i*2)//4
    col = (i*2)%4
    
    #show image
    path = Path(tgs+"/train/images") / '{}.png'.format(imgId)
    img = imread(path)
    ax[row, col].imshow(img)
    
    #Show mask
    path = Path(tgs+"/train/masks") / '{}.png'.format(imgId)
    img = imread(path)
    ax[row, col + 1].imshow(img, cmap = "gray")

### Function which takes id of an image, reads it and returns ndarray

In [ ]:
tgs = "../input"
tgs1 = "../input/tgs-salt-identification-challenge"
def getImage(imgId):
    path = Path(tgs+"/train/images/") / '{}'.format(imgId)
    img = imread(path)
    return img.astype(np.uint8)

def getGrayImage(imgId):
    path = Path(tgs+"/train/images/")/'{}'.format(imgId)
    img = imread(path).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return img

def getGrayTestImage(imgId):
    path = Path(tgs+"/test/images/")/'{}'.format(imgId)
    img = imread(path).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return img

def getMask(imgId):
    path = Path(tgs+"/train/masks/") / '{}'.format(imgId)
    img = imread(path)
    return img.astype(np.uint8)

### Define Accuracy Metric
[IoU Metric Explanation](https://www.kaggle.com/pestipeti/explanation-of-scoring-metric)

In [ ]:
thresholds = [0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
n_thresholds = len(thresholds)

#function to return A∩B and A∪B
def IoUhelper(TrueMask, predictedMask):
    intersection = cv2.bitwise_and(TrueMask, predictedMask)
    union = cv2.bitwise_or(TrueMask, predictedMask)
    intersectionCnt = cv2.countNonZero(intersection)
    unionCnt = cv2.countNonZero(union)
    return intersectionCnt, unionCnt

def meanHit(TrueMask, predictedMask):
    hitCnt = 0
    intersectionCnt, unionCnt = IoUhelper(TrueMask, predictedMask)
#     print("intersction = ", intersectionCnt, "unionCnt = ", unionCnt)
    #if both TrueMask is empty and PredictedMask is empty
    if(intersectionCnt == 0 and unionCnt == 0):
        return 1
    
    #if TrueMask in empty and  Predicted mask in non empty
    #--------------------OR-------------------------
    #if TrueMask is non empty and Predicted Mask is empty
    if(intersectionCnt == 0 and unionCnt != 0):
        return 0
    
    #if TrueMask is non empty and predicted Mask is non emtpy
    IoU = intersectionCnt/unionCnt
    for t in thresholds:
        hitCnt+=int(IoU > t)
    return hitCnt/n_thresholds

def accuracy(mask, mask_pred):
    num = mask.shape[0]
    scores = []
    for i in range(num):
        score = meanHit(mask[i], mask_pred[i])
        scores.append(score)
        
    return np.mean(scores)

# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    y_pred_ = tf.to_int32(y_pred > 0.55)
    y_true = tf.to_int32(y_true > 0.55)
    for t in np.arange(0.5, 1.0, 0.05):
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def get_iou_vector(A, B):
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch]>0, B[batch]>0
#         if np.count_nonzero(t) == 0 and np.count_nonzero(p) > 0:
#             metric.append(0)
#             continue
#         if np.count_nonzero(t) >= 1 and np.count_nonzero(p) == 0:
#             metric.append(0)
#             continue
#         if np.count_nonzero(t) == 0 and np.count_nonzero(p) == 0:
#             metric.append(1)
#             continue
        
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        iou = (np.sum(intersection > 0) + 1e-10 )/ (np.sum(union > 0) + 1e-10)
        thresholds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in thresholds:
            s.append(iou > thresh)
        metric.append(np.mean(s))

    return np.mean(metric)

def my_iou_metric(label, pred):
    return -tf.py_func(get_iou_vector, [label, pred>0.5], tf.float64)

## intersection over union
def IoU(y_true, y_pred, eps=1e-6):
    if np.max(y_true) == 0.0:
        return IoU(1-y_true, 1-y_pred) ## empty image; calc IoU of zeros
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    return -K.mean( (intersection + eps) / (union + eps), axis=0)

#### Checking correctness of accuracy metric

In [ ]:
# Testing accuracy metric
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(12, 4)
#Read an image with mask
imgId = train[haveMask].index[4]
img1 = getImage(imgId+".png")
img2 = getMask(imgId+".png")
img3 = (np.ones(img2.shape)*255).astype(np.uint8)

ax[0].imshow(img1)
ax[0].set_title("Image")

ax[1].imshow(img2, cmap = 'gray')
ax[1].set_title("True Mask")

ax[2].imshow(img3, cmap = "gray", vmin = 0, vmax = 255)
ax[2].set_title("Predicted Mask")
print("Average Precision = ",meanHit(img2, img3))

### rle_encoder
> Function takes a mask and return string which contains rle_encoding<br>
 Funtion expects that mask only contains 0 and 255 as pixel value

In [ ]:
def rle_encoding(mask):
    mask = mask.ravel()
    encoding = ""
    i = 0

    while(i<len(mask)):
        currCnt = 0
        start = i
        if(mask[i] == 255):
            while(i < len(mask) and mask[i] == 255):
                currCnt+=1
                i+=1
                
            encoding+=(" "+ str(start+1) + " " + str(currCnt))
        else: i+=1
    return encoding.strip()

#### Checking Correctness of Encoding Function
- Note that encoding is done by taking transpose of image

In [ ]:
imgId = train[haveMask].index[18]
rle_orig = train[haveMask].rle_mask[18]
img1 = getImage(imgId+".png")
img2 = getMask(imgId+".png")
rle1 = rle_encoding(np.transpose(img2))
print("Original Encoding\n", rle_orig)
print("Len1 = ", len(rle_orig.split()))
print("-------")
print("Encoding\n", rle1)
print("Len2 = ", len(rle1.split()))

### Read Train and Test Image 

In [ ]:
#Get the Directory
TRAIN_IMAGE_DIR = tgs+'/train/images'
TRAIN_MASK_DIR = tgs+'/train/masks'
TEST_MASK_DIR = tgs+'/test/images'

im_height = 128
im_width = 128

train_image_list = os.listdir(TRAIN_IMAGE_DIR)
train_mask_list = os.listdir(TRAIN_MASK_DIR)
test_image_list = os.listdir(TEST_MASK_DIR)

In [ ]:
X_train_image = np.zeros((len(train_image_list), 
                         im_height, im_width, 1), dtype = np.uint8)
Y_train_mask = np.zeros((len(train_mask_list),
                         im_height, im_width, 1), dtype = np.uint8)

for i in tqdm(range(len(train_image_list))):
    imgId = train_image_list[i]
    img = getGrayImage(imgId)
    img = cv2.resize(img, (im_height, im_width))
    X_train_image[i] = img.reshape(im_height, im_width, 1)

    img = getMask(imgId)
    img = cv2.resize(img, (im_height, im_width))
    Y_train_mask[i] = img.reshape(im_height, im_width, 1)
    
print("X_train Shape = ", X_train_image.shape, "Y_train Shape = ", Y_train_mask.shape)

#####  Check if Data is loaded properly

In [ ]:
# os.listdir("./")


In [ ]:
fig, ax = plt.subplots(1, 2, sharex = "col", sharey= "row")
fig.set_size_inches(8, 4)

img = X_train_image[70, :, :, 0]
ax[0].imshow(img, cmap = 'gray')
ax[1].imshow(Y_train_mask[70, :,:,0], cmap = "gray")

### Data Augmentation ( Basic Augmentation)

In [ ]:
temp = np.array([[[[1, 3], [1, 2],[0, 2]]
                ,[[0, 4], [0, 5],[2, 1]]],
                [[[1, 2], [1, 6],[0, 12]]
                ,[[1, 30], [1, 21],[1, 123]]],
                [[[1, 90], [1, 91],[0, 32]]
                ,[[1, 56], [1, 72],[1, 97]]]])
# print(temp, "\n---")
# print(np.sum(temp, axis = 1), np.sum(temp, axis = 3).shape)
# print(temp.shape)
# # temp = np.append(temp, [[[[1, 10], [1, 11],[0, 70]]
# #                 ,[[1, 67], [1, 43],[4, 324]]]], axis = 0)
# print(temp.shape)
# print(temp)

In [ ]:
# X_train_image1 = np.append(X_train_image, [np.fliplr(x) for x in X_train_image], axis = 0)
# Y_train_mask1 = np.append(Y_train_mask, [np.fliplr(y) for y in Y_train_mask], axis = 0)
# X_train_image = np.append(X_train_image1, [np.flipud(x) for x in X_train_image], axis = 0)
# Y_train_mask = np.append(Y_train_mask1, [np.flipud(x) for x in Y_train_mask], axis = 0)
# print(X_train_image.shape)
# print(Y_train_mask.shape)

#### Train Validation Split



##### Step1.Count One(255) is a mask which will be used for assigning label to an image
> Labels will be based on how many ones(or255) are there in a mask. 

In [ ]:
temp = np.array([[[[1], [1],[0]]
                ,[[0], [0],[2]]],
                [[[1], [1],[0]]
                ,[[1], [1],[1]]],
                [[[1], [1],[0]]
                ,[[1], [1],[1]]]])
# print(temp[0])
print(temp,"\n", temp.shape)

np.sum(np.count_nonzero(temp, axis = 2
                ), axis = 1)

In [ ]:
print(X_train_image.dtype, Y_train_mask.dtype, X_train_image.shape, Y_train_mask.shape)
cntOne = np.sum(np.count_nonzero(Y_train_mask, axis = 2), axis = 1).ravel()
print(cntOne.shape)

In [ ]:
#Checking correctness
print(np.count_nonzero(Y_train_mask[2]))
print(cntOne[2])

##### Step2. Create Labels

In [ ]:
#Testing how to assign labels
x = np.array([0, 1.0, 3.0, 1.6])
bins = np.array([0, 1.0, 2.5, 4.0, 10.0])
inds = np.digitize(x, bins)
np.unique(inds)

In [ ]:
#Create Labels
bins = np.array([0, 1600, 3200, 4800, 6400, 8000, 9600, 11200, 12800, 14400])
labels = np.digitize(cntOne, bins)
print(np.unique(labels), labels.shape)

##### Step3. Create Split

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(X_train_image, Y_train_mask,
                                                  test_size = 0.1,
                                                 random_state = 43, 
                                                 stratify = labels)
print("TrainX shape = ", train_X.shape, "TrainY Shape = ", train_Y.shape)
print("ValidationX Shape = ", val_X.shape, "ValidationY Shape = ", val_Y.shape)

### Augmentation

In [ ]:
def augment(X, Y):
    print("FlipLR")
    X1 = np.append(X, [np.fliplr(x) for x in X], axis = 0)
    Y1 = np.append(Y, [np.fliplr(y) for y in Y], axis = 0)
    
    print("Roll")
    X = np.append(X1, [np.roll(x, 40, axis = 1) for x in X1], axis = 0)
    Y = np.append(Y1, [np.roll(y, 40, axis = 1) for y in Y1], axis = 0)

    m = X.shape[0]
    np.random.seed(2000)
    index = np.arange(0, m)
    np.random.shuffle(index)
    print("Shuffle")
    X = X[index]
    Y = Y[index]
    return X, Y

print("Training Augmentation")
train_X, train_Y = augment(train_X, train_Y)
val_X, val_Y = augment(val_X, val_Y)
print(train_X.shape, train_Y.shape)
print(val_X.shape, val_Y.shape)



### Normalize

In [ ]:
print(train_Y[70, :, :, 0])
train_X = (train_X.astype(np.float32)/255.0)
train_Y = (train_Y/255.0).astype(np.bool).astype(np.uint8)
val_X = (val_X.astype(np.float32)/255.0)
val_Y = (val_Y/255.0).astype(np.bool).astype(np.uint8)
print(train_X.dtype, train_Y.dtype)
print(train_Y[70, :, :, 0])

#### Training using Convolutional Neural Network

In [ ]:
import keras

# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU':8} ) 
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *

In [ ]:
def batchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def residualBlock(blockInput, numChannel, matchChannel = False):
    #inputs --> BN --> Relu
    x = batchActivate(blockInput)
    #blockInput --> BN --> Relu --> weights
    x = Conv2D(numChannel, (3, 3), activation= None, padding = "same",
               use_bias = False)(x)
    #blockInput --> BN --> Relu --> weights --> BN --> Relu
    x = batchActivate(x)
    #blockInput --> BN --> Relu --> weights --> BN --> Relu --> weights
    x = Conv2D(numChannel, (3, 3), activation=None, padding = "same", 
              use_bias = False)(x)
    
    #Match Channels
    if matchChannel:
        blockInput = Conv2D(numChannel, (1, 1), activation= None, padding = "same", 
                           use_bias = False)(x)
    
    #blockInput----------------------------------------------------|
    #                                                              +
    #                                                              |
    #blockInput --> BN --> Relu --> weights --> BN --> Relu --> weights
    x = Add()([x, blockInput])
    #Not sure about this layer
#     x = Activation('relu')(x)
    return x

In [ ]:
inputs = Input(shape = (im_height, im_width, 1))

#Initaial layer
init = Conv2D(16, (7, 7), activation="relu", padding = "same")(inputs)
#Pending:One pool layer

#128, 128
conv1 = residualBlock(init, 16)
conv1 = residualBlock(conv1, 16)
c1 = residualBlock(conv1, 16)
c1 = BatchNormalization()(c1)
# c1 = Dropout(0.2)(c1)
p1 = MaxPooling2D(pool_size=(2, 2))(c1)

#64, 64
conv2 = residualBlock(p1, 32, True)
c2 = residualBlock(conv2, 32)
c2 = BatchNormalization()(c2)
# c2 = Dropout(0.2)(c2)
p2 = MaxPooling2D(pool_size = (2, 2))(c2)
# p2 = Dropout(0.1)(p2)

#32, 32
conv3 = residualBlock(p2, 64, True)
c3 = residualBlock(conv3, 64)
c3 = BatchNormalization()(c3)
# c3 = Dropout(0.2)(c3)
p3 = MaxPooling2D(pool_size = (2, 2))(c3)
# p3 = Dropout(0.1)(p3)

#16, 16
conv4 = residualBlock(p3, 128, True)
c4 = residualBlock(conv4, 128)
c4 = BatchNormalization()(c4)
# c4 = Dropout(0.2)(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)
# p4 = Dropout(0.1)(p4)

#8, 8
conv5 = residualBlock(p4, 256, True)
conv5 = residualBlock(conv5, 256)
conv5 = residualBlock(conv5, 256)
c5 = residualBlock(conv5, 256)
c5 = BatchNormalization()(c5)
# p5 = MaxPooling2D(pool_size=(2, 2))(conv5)


# Start decoding-----------------
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), activation = "relu",
                    padding = "same")(c5)
#-----
#16, 16
u6 = concatenate([u6, c4])
conv6 = residualBlock(u6, 128, True)
conv6 = residualBlock(conv6, 128)
#------
u7 = Conv2DTranspose(64, (2, 2), strides = (2, 2,), activation = "relu",
                     padding = "same")(conv6)
#32, 32
u7 = concatenate([u7, c3])
conv7 = residualBlock(u7, 64, True)
conv7 = residualBlock(conv7, 64)
#------
u8 = Conv2DTranspose(32, (2, 2), strides = (2, 2), activation = "relu",
                     padding = "same")(conv7)
#64, 64
u8 = concatenate([u8, c2])
conv8 = residualBlock(u8, 32, True)
conv8 = residualBlock(conv8, 32)

#---
u9 = Conv2DTranspose(16, (2, 2), strides = (2, 2), activation = "relu",
                     padding = "same")(conv8)
#128, 128
u9 = concatenate([u9, c1], axis=3)
conv9 = residualBlock(u9, 16, True)
conv9 = residualBlock(conv9, 16)
conv9 = residualBlock(conv9, 16)

outputs1 = Conv2D(16, (1, 1), activation = "relu", padding = "same")(conv9)
outputs = Conv2D(1, (1, 1), activation = "sigmoid")(outputs1)

model = Model(inputs = [inputs], outputs = [outputs])
optimizer = keras.optimizers.Adam()

model.compile(optimizer = optimizer, loss = "binary_crossentropy", 
              metrics = [my_iou_metric])
model.summary()

In [ ]:
import keras
earlyStopping = EarlyStopping(patience = 20, verbose = 1)
checkpointer = ModelCheckpoint('model-tgs-salt-1.h5',monitor = 'val_my_iou_metric', 
                               verbose=1, save_best_only=True)
reducelr=ReduceLROnPlateau(monitor='val_my_iou_metric',patience=5, 
                            min_lr=0.00001, verbose=1,factor=0.5)
epochs = 50

batch_size = 64

history = model.fit(train_X , train_Y, epochs = epochs,
                    batch_size = batch_size,
                    validation_data = (val_X, val_Y),
                    callbacks=[earlyStopping, checkpointer, reducelr])

In [ ]:
model1 = load_model("./model-tgs-salt-1.h5",
                   custom_objects = {'my_iou_metric':my_iou_metric})

In [ ]:
history.history

In [ ]:
lr = history.history['lr']
x = -1
lr1 = []
lrepoch = []
for (n,r) in enumerate(lr):
    if r!=x:
        lr1.append(r)
        lrepoch.append(n + 1)
        x = r

In [ ]:
lrepoch

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(2, 1, sharex = 'col', sharey = 'row')
fig.set_size_inches(15, 16)  #width, height

ax_loss.plot(history.history['loss'], label = 'train_loss')
ax_loss.plot(history.history['val_loss'], label = 'val_loss')
# ax_loss.plot(history.history['lr'], label = 'learning_rate')
ax_loss.legend()
plt.xticks(lrepoch)
ax_loss.grid(True, which = 'both')
ax_loss.set_xlabel("num_epoch")
ax_loss.set_ylabel("loss/lr")
ax_loss.set_title("Loss")
ax_acc.plot(history.history['my_iou_metric'], label = 'train_metric')
ax_acc.plot(history.history['val_my_iou_metric'], label = "val_metric")
# ax_acc.plot(history.history['lr'], label = 'learning_rate')
ax_acc.legend()
ax_acc.grid(True, which = 'both')
plt.xticks(lrepoch)
ax_acc.set_xlabel("num_epoch")
ax_acc.set_ylabel("acc/lr")
ax_acc.set_title("iou metric")

### Predict model on some random validation set images

In [ ]:
print(X_train_image[12:13, :,:, 0].shape)

In [ ]:
samples = train[haveMask].sample(8)
fig, ax = plt.subplots(3, 3, sharex = "col", sharey = "row")
fig.set_size_inches(9, 9)

index = np.random.randint(1, 400, size = (3))

for i,ind in enumerate(index):
    img = val_X[ind:ind+1, :, :]
    origMask = val_Y[ind:ind+1, :, :]
    predMask = model1.predict(img)
    ax[i, 0].imshow(img[0, :, :, 0], cmap = 'gray')
    ax[i, 0].set_title(str(ind))
    ax[i, 1].imshow(origMask[0, :, :, 0], cmap = 'gray')
    ax[i, 2].imshow(predMask[0, :, :, 0]>0.55
                    , cmap = 'gray')
    origMask = cv2.inRange(origMask[0, :, :, 0], 0.55, 255)
    predMask = cv2.inRange(predMask[0, :, :, 0], 0.55, 255)
    ax[i, 1].imshow(origMask, cmap = 'gray')
    ax[i, 2].imshow(predMask, cmap = 'gray')
    print("mean IoU = ", meanHit(origMask, predMask))

# pred_mask = model.predict(X_train_image[22:23, :, :])
# print(img.shape)
# print(img[0,:,:])
# plt.imshow(img[0,:,:,0]>0.5, cmap = "gray")

#### mean IoU on Validation Set

In [ ]:
totIoU = 0.0
m = val_X.shape[0]
# print("m = ", m)
for i in tqdm(range(m)):
    img = val_X[i:i+1, :, :]
    origMask = val_Y[i:i+1, :, :]
    predMask = model1.predict(img)
    origMask = cv2.inRange(origMask[0, :, :, 0], 0.55, 255)
    predMask = cv2.inRange(predMask[0, :, :, 0], 0.55, 255)
    totIoU += meanHit(origMask, predMask)
# print("totIoU=", totIoU)
print("mean IoU = ", totIoU/m)

---

#### Load Test Image Data

In [ ]:
X_test_image = np.zeros((len(test_image_list), im_height, 
                       im_width, 1), dtype = np.uint8)
for i in tqdm(range(len(test_image_list))):
    imgId = test_image_list[i]
    img = getGrayTestImage(imgId)
    img = cv2.resize(img, (im_height, im_width))
    X_test_image[i] = img.reshape(im_height, im_width, 1)

In [ ]:
X_test_image.shape

In [ ]:
#Normalize 
X_test_image = X_test_image/255.0

#### Predictiion on some random test images

In [ ]:
# Y_test = model.predict(X_test_image[14:15,:,:])
fig, ax = plt.subplots(3, 2, sharex = "col", sharey = "row")
fig.set_size_inches(12, 12)

index = np.random.randint(1, 18000, size = (3))

for i,ind in enumerate(index):
    img = X_test_image[ind:ind+1, :, :]
    predMask = model1.predict(img)
    ax[i, 0].imshow(img[0, :, :, 0], cmap = 'gray')
    predMask = cv2.inRange(predMask[0, :, :, 0], 0.5, 255)   #type uint8 shape:128,128
    print(predMask.dtype)
    ax[i, 1].imshow(predMask, cmap = 'gray')

#### Prepare submission file

In [ ]:
import gc
gc.collect()

In [ ]:
print(X_test_image[0].dtype
     )

In [ ]:
test_image_list[0][:-4]

In [ ]:
origSize = (101, 101)
submit_names = []
submit_rleMasks = []

for i in tqdm(range(len(test_image_list))):
    img = X_test_image[i:i+1,:,:]   #1, 128, 128, 1
    predMask =  model1.predict(img)
    predMask = cv2.inRange(predMask[0, :, :, 0], 0.55, 255)  #128, 128, uint8
    predMask = cv2.resize(predMask, origSize)
    submit_names.append(test_image_list[i][:-4])
    submit_rleMasks.append(rle_encoding(np.transpose(predMask)))

In [ ]:
sub = pd.DataFrame({'id':submit_names, 'rle_mask': submit_rleMasks})
print(sub.shape)
sub.head()

In [ ]:
#Replace emtpy rle_mask with np.nan
for i in tqdm(range(len(test_image_list))):
    if(len(sub.iloc[i, 1]) == 0):
        sub.iloc[i, 1] = np.nan

In [ ]:
print(sub.shape)
sub.head(n = 10)

In [ ]:
# img = getGrayTestImage("604e99b0c6.png")
print(img.shape)
ind = 213
fig ,ax = plt.subplots(1, 2, sharex = "col", sharey = "row")
img = X_test_image[ind:ind+1, :, :]
ax[0].imshow(img[0, :, :, 0], cmap = "gray")
predMask = model1.predict(img)
ax[1].imshow(predMask[0,:,:,0]>0.55, cmap = "gray")

In [ ]:
sub.to_csv("tgsModel1.csv", index = False)